<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#数据集的特征提取类" data-toc-modified-id="数据集的特征提取类-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>数据集的特征提取类</a></span><ul class="toc-item"><li><span><a href="#线性链条件随机场类" data-toc-modified-id="线性链条件随机场类-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>线性链条件随机场类</a></span></li><li><span><a href="#使用较小的训练数据集进行训练" data-toc-modified-id="使用较小的训练数据集进行训练-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>使用较小的训练数据集进行训练</a></span></li><li><span><a href="#载入训练好的-CRF-模型，并在较小的测试数据集上进行测试" data-toc-modified-id="载入训练好的-CRF-模型，并在较小的测试数据集上进行测试-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>载入训练好的 CRF 模型，并在较小的测试数据集上进行测试</a></span></li><li><span><a href="#使用完整的训练数据集进行训练" data-toc-modified-id="使用完整的训练数据集进行训练-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>使用完整的训练数据集进行训练</a></span></li></ul></li><li><span><a href="#第-2-个版本（优化）数据集的特征提取类" data-toc-modified-id="第-2-个版本（优化）数据集的特征提取类-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>第 2 个版本（优化）数据集的特征提取类</a></span><ul class="toc-item"><li><span><a href="#定义-scipy.optimize-的-L-BFGS-函数所需要的回调函数" data-toc-modified-id="定义-scipy.optimize-的-L-BFGS-函数所需要的回调函数-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>定义 scipy.optimize 的 L-BFGS 函数所需要的回调函数</a></span></li><li><span><a href="#使用较小的训练数据集进行训练" data-toc-modified-id="使用较小的训练数据集进行训练-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>使用较小的训练数据集进行训练</a></span></li><li><span><a href="#使用完整的训练数据集进行训练" data-toc-modified-id="使用完整的训练数据集进行训练-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>使用完整的训练数据集进行训练</a></span></li></ul></li></ul></div>

In [1]:
import numpy as np
from collections import Counter

## 数据集的特征提取类

In [2]:
class FeatureSet:
    def __init__(self):
        self.label_ids = {'*':0} # 存放标签及其 ID。缺省：开始标签及其 ID
        # 存放子特征字符串、前 1 个标签 ID-当前标签 ID、子特征 ID，
        # 形成字典的字典结构。后面 2 个形成内部字典的 key-value
        self.subfeature_yy_ids = {}
        # 对子特征的出现次数计数， key：子特征 ID； value：出现次数
        self.subfeature_counters = Counter()
        self.scale_threshold = 1e250 # 用于计算结果溢出处理

    #  读取 CONLL 格式的语料文件
    def ReadCorpusCONLL(self, filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
        self.data = [([], [])] # 分别用来存放 TOKEN_POS 序列及其 LABEL
        for line in lines:
            words = line.strip().split()
            if words == []: # 遇到空行
                if self.data[-1] != ([], []):
                    self.data.append(([], []))
            else:
                self.data[-1][0].append(words[:-1]) # 添加 TOKEN 和 POS
                self.data[-1][1].append(words[-1]) # 添加 LABEL
        if self.data[-1] == ([], []):
            del(self.data[-1])

    # 为句子生成子特征集，这是缺省的特征函数
    #sentence: [[TOKEN1, POS1], [TOKEN2, POS2], ...]
    def Featurize(self, sentence):
        sentence_subfeatures, T = [], len(sentence)
        for t in range(T):
            token_features = []
            token_features.append('U[0]:%s' % sentence[t][0]) # 第 t 步 TOKEN
            # 第 t 步 POS(Part-Of-Speech)
            token_features.append('POS_U[0]:%s' % sentence[t][1])
            if t < T-1:
                # 第 t+1 步 TOKEN
                token_features.append('U[+1]:%s' % (sentence[t+1][0]))
                # 第 t 和 t+1 步 TOKEN
                token_features.append('B[0]:%s %s' % (sentence[t][0], sentence[t+1][0])) 
                # 第 t+1 步 POS
                token_features.append('POS_U[1]:%s' % sentence[t+1][1])
                # 第 t 和 t+1 步 POS
                token_features.append('POS_B[0]:%s %s' % (sentence[t][1], sentence[t+1][1])) 
                if t < T-2:
                    # 第 t+2 步 TOKEN
                    token_features.append('U[+2]:%s' % (sentence[t+2][0]))
                    # 第 t+2 步 POS
                    token_features.append('POS_U[+2]:%s' % (sentence[t+2][1]))
                    # 第 t+1 和 t+2 步 POS
                    token_features.append('POS_B[+1]:%s %s' % (sentence[t+1][1], sentence[t+2][1])) 
                    # 第 t、 t+1 和 t+2 步 POS
                    token_features.append('POS_T[0]:%s %s %s' % (sentence[t][1], sentence[t+1][1], sentence[t+2][1])) 
            if t > 0:
                # 第 t-1 步 TOKEN
                token_features.append('U[-1]:%s' % (sentence[t-1][0]))
                token_features.append('B[-1]:%s %s' % (sentence[t-1][0], sentence[t][0])) # 第 t-1 和 t 步 TOKEN
                # 第 t-1 步 POS
                token_features.append('POS_U[-1]:%s' % (sentence[t-1][1]))
                token_features.append('POS_B[-1]:%s %s' % (sentence[t-1][1], sentence[t][1])) # 第 t-1 和 t 步 POS
                if t < T-1:
                # 第 t-1、 t 和 t+1 步 POS
                    token_features.append('POS_T[-1]:%s %s %s' % (sentence[t-1][1], sentence[t][1], sentence[t+1][1]))
                if t > 1:
                    # 第 t-2 步 TOKEN
                    token_features.append('U[-2]:%s' % (sentence[t-2][0]))
                    # 第 t-2 步 POS
                    token_features.append('POS_U[-2]:%s' % (sentence[t-2][1]))
                    # 第 t-2 和 t-1 步 POS
                    token_features.append('POS_B[-2]:%s %s' % (sentence[t-2][1], sentence[t-1][1])) 
                    # 第 t-2、 t-1 和 t 步 POS
                    token_features.append('POS_T[-2]:%s %s %s' % (sentence[t-2][1], sentence[t-1][1], sentence[t][1])) 
            sentence_subfeatures.append(token_features)
        return sentence_subfeatures

    def Add(self, y_prev, y, subfeatures):
        for subfeature in subfeatures: # 遍历每个子特征
            if subfeature in self.subfeature_yy_ids.keys(): # 子特征，已存在
                # 标签对，已存在
                if (y_prev, y) in self.subfeature_yy_ids[subfeature].keys():
                    self.subfeature_counters[self.subfeature_yy_ids[subfeature][(y_prev, y)]] += 1
                else: # 标签对，不存在，新增
                    new_id = len(self.subfeature_counters)
                    self.subfeature_yy_ids[subfeature][(y_prev, y)] = new_id
                    self.subfeature_counters[new_id] = 1
                #(-1, y) 表示当前标签 (y 的当前状态)
                if (-1, y) in self.subfeature_yy_ids[subfeature].keys():
                    self.subfeature_counters[self.subfeature_yy_ids[subfeature][(-1, y)]] += 1
                else: # 不存在，新增
                    new_id = len(self.subfeature_counters)
                    self.subfeature_yy_ids[subfeature][(-1, y)] = new_id
                    self.subfeature_counters[new_id] = 1
            else: # 子特征，不存在
                self.subfeature_yy_ids[subfeature] = {}
                #Bigram feature
                new_id = len(self.subfeature_counters)
                self.subfeature_yy_ids[subfeature][(y_prev, y)] = new_id
                self.subfeature_counters[new_id] = 1
                # Unigram feature
                new_id = len(self.subfeature_counters)
                self.subfeature_yy_ids[subfeature][(-1, y)] = new_id
                self.subfeature_counters[new_id] = 1

    # 为语料数据生成所有的子特征
    def GenerateAllSubfeatures(self):
        self.sentences_subfeatures = []
        for token_poses, labels in self.data: # 遍历每条句子（语料数据）
            sentence_subfeatures = self.Featurize(token_poses)
            self.sentences_subfeatures.append(sentence_subfeatures)
            y_prev = 0 # 开始标签'*' 的 ID 号
            for t in range(len(token_poses)): # 遍历每个单词（语料数据）
                try:
                    y = self.label_ids[labels[t]]
                except KeyError: # 当前标签及其 ID 号不存在，新增 1 个
                    y = len(self.label_ids) # 新 ID 号
                    self.label_ids[labels[t]] = y # 新增标签及其 ID 号
                self.Add(y_prev, y, sentence_subfeatures[t]) # 统计子特征数据
                y_prev = y
        self.weights = np.zeros(len(self.subfeature_counters))
        # 数组：保存每个子特征的出现次数
        self.empirical_counts = np.zeros(len(self.subfeature_counters))
        for id, counts in self.subfeature_counters.items():
            self.empirical_counts[id] = counts
        # 按句子存放 (y_prev, y) 到 ids 的映射关系
        self.yy_ids = []
        for sentence_subfeatures in self.sentences_subfeatures: # 遍历每个句子
            sentence_yy_ids = []
            for t in range(len(sentence_subfeatures)): # 遍历每个 TOKEN
                token_yy_ids = {}
                for subfeature in sentence_subfeatures[t]: # 遍历 TOKEN 的每个子特征
                    for (y_prev, y), id in self.subfeature_yy_ids[subfeature].items():
                        if (y_prev, y) in token_yy_ids.keys():
                            token_yy_ids[(y_prev, y)].add(id)
                        else:
                            token_yy_ids[(y_prev, y)] = {id}
                sentence_yy_ids.append([((y_prev, y), ids) for (y_prev, y), ids in token_yy_ids.items()])
            self.yy_ids.append(sentence_yy_ids)
         # 生成 id-labels 数组，用于路径回溯
        self.id_labels = ['?']*len(self.label_ids) # 初始化
        for label, id in self.label_ids.items():
            self.id_labels[id] = label

    # 计算给定句子的权值-特征的点积（在 K 个特征函数上求和）
    def CalcWeightDotFeature(self, subfeature_yy_ids, label_ids, sentence_subfeatures, weights):
        T, M = len(sentence_subfeatures), len(label_ids)
        table = np.zeros((T, M, M))
        for t in range(T): # 遍历每个 TOKEN-POS
            for subfeature in sentence_subfeatures[t]: # 遍历每个 TOKEN-POS 的子特征
                try:
                    for (y_prev, y), id in subfeature_yy_ids[subfeature].items():
                        if y_prev != -1: #Bigram feature
                            table[t, y_prev, y] = table[t, y_prev, y] + weights[id]
                        else: #Unigram feature
                            table[t, :, y] = table[t, :, y] + weights[id]
                except KeyError: # 在使用测试数据集进行测试时，可能遇到不存在的子特征
                    pass
            table[t] = np.exp(table[t]) # 权值-特征的势函数
            # 清除不存在的特征
            if t == 0:
                table[t, 1:] = 0
            else:
                table[t, :, 0] = 0
                table[t, 0, :] = 0
        return table

    # 为数据集生成所有的 M 矩阵
    def GeneratePotentialMatrix(self, weights):
        self.Ms = []
        for sentence_subfeatures in self.sentences_subfeatures: # 遍历每个句子
            self.Ms.append(self.CalcWeightDotFeature(self.subfeature_yy_ids,self.label_ids, sentence_subfeatures, weights))

    # 计算给定句子的 alpha、 beta 与 Z
    def CalcForwardBackward(self, i_sentence, sentence_subfeatures):
        T, M = len(sentence_subfeatures), len(self.label_ids)
        #scales 用于计算结果溢出处理
        alphas, betas, scales = np.zeros((T, M)), np.zeros((T, M)), {}
        alphas[0] = self.Ms[i_sentence][0][0, :] # 初始化 alpha 向量
        t = 1
        while t < T: # 递推计算前向概率
            overflow = False
            for id in range(1, M):
                alphas[t, id] = np.dot(alphas[t-1, :], self.Ms[i_sentence][t][:, id])
                if alphas[t, id] > self.scale_threshold: # 结果溢出
                    overflow = True
                    scales[t-1] = self.scale_threshold
                    break
            if overflow: # 溢出，计算结果的处理
                alphas[t-1], alphas[t] = alphas[t-1] / self.scale_threshold, 0
            else:
                t += 1
        betas[T-1] = 1.0 # 初始化 beta 向量
        for t in range(T-2, -1, -1): # 递推计算后向概率
            betas[t] = np.dot(betas[t+1].reshape(1, -1), self.Ms[i_sentence][t+1].T)
            if t in scales.keys():
                betas[t] = betas[t] / scales[t]
        return alphas, betas, sum(alphas[T-1]), scales

    # 为数据集生成所有的 alpha、 beta 向量与 Z
    def GenerateForwardBackward(self):
        self.alphas, self.betas, self.Zs, self.scales = [], [], [], []
        # 遍历每个句子
        for i_sentence, sentence_subfeatures in enumerate(self.sentences_subfeatures):
            alphas, betas, Z, scales = self.CalcForwardBackward(i_sentence,
            sentence_subfeatures)
            self.alphas.append(alphas)
            self.betas.append(betas)
            self.Zs.append(Z)
            self.scales.append(scales)

In [3]:
 from math import log

In [4]:
# 计算数据集的损失函数（负对数似然函数）
def LossFunction(weights, *args):
    feature_set, squared_sigma, step = args # 对象、正则化参数
    step[0] += 1
    feature_set.GeneratePotentialMatrix(weights) # 使用新权值，更新 M 矩阵
    feature_set.GenerateForwardBackward() # 使用新权值，更新 alpha、 beta 和 Z
    logZ_sum = sum([log(Z) for Z in feature_set.Zs]) + sum([log(scale) for scales in feature_set.scales for t, scale in scales.items()])
    # 数据集的对数似然函数 logP(y1|x1)+...，每个单样本的对数似然函数为： sum(w*f)-logZ，
    # 所有样本的对数似然函数为： sum(sum(w*f))-logZ_sum
    # 最后一项是正则化项
    loglikelihood = np.dot(feature_set.empirical_counts, weights) - logZ_sum - np.dot(weights, weights)/(squared_sigma*2)
    loss = -loglikelihood
    # 计算梯度（其中， logZ_sum 对 weights 的梯度，都归结为此项。
    # 先考虑单样本情形， logZ 求偏导 =>F(有效)*P(y|x)，即下面的 prob 项；
    # 再考虑多样本情形即可）
    expected_counts = np.zeros(len(feature_set.subfeature_counters))
    for i_sentence, sentence_yy_ids in enumerate(feature_set.yy_ids): # 每个句子
        for i_token, token_yy_ids in enumerate(sentence_yy_ids): # 每个 TOKEN
            # 在相应的特征（有效）处， +p(prev,y|X,t)
            for (y_prev, y), ids in token_yy_ids:
                if y_prev == -1: # 单状态
                    if i_token in feature_set.scales[i_sentence].keys(): # 结果溢出过
                        prob = (feature_set.alphas[i_sentence][i_token, y] * feature_set.betas[i_sentence][i_token, y]) *feature_set.scales[i_sentence][i_token] / feature_set.Zs[i_sentence]
                    else:
                        prob = (feature_set.alphas[i_sentence][i_token, y] *feature_set.betas[i_sentence][i_token, y]) / feature_set.Zs[i_sentence]
                elif i_token == 0: # 初始状态
                    if y_prev != 0: # 需要判断，否则出现 overflow
                        continue
                    prob = feature_set.Ms[i_sentence][i_token][0, y] *feature_set.betas[i_sentence][i_token, y] / feature_set.Zs[i_sentence]
                else: # 双状态
                    if y_prev == 0 or y == 0: # 需要判断，否则出现 overflow
                        continue
                    prob = feature_set.alphas[i_sentence][i_token-1, y_prev] * feature_set.Ms[i_sentence][i_token][y_prev, y] * feature_set.betas[i_sentence][i_token, y] / feature_set.Zs[i_sentence]
                for id in ids: # 特征有效处， +prob
                    expected_counts[id] = expected_counts[id] + prob
    gradients = feature_set.empirical_counts - expected_counts - weights/squared_sigma
    gradients = -gradients
    print('Step', step[0], 'Loss:', loss)
    return loss, gradients

In [5]:
import os
import pickle
from scipy.optimize import fmin_l_bfgs_b
import time
import datetime

### 线性链条件随机场类

In [6]:
class LinearChainCRF:
    def __init__(self, filename):
        self.filename = filename
        self.squared_sigma = 10.0 # 正则化参数
        self.feature_set = FeatureSet()

    def fit(self):
        print('Reading corpus data ...')
        self.feature_set.ReadCorpusCONLL(self.filename)
        self.feature_set.GenerateAllSubfeatures()
        print('Data reading completed!')
        start_time = time.time()
        print('[%s] Starting to train ...' % datetime.datetime.now())
        self.feature_set.weights, self.loss, self.info = fmin_l_bfgs_b(func = LossFunction, x0 = self.feature_set.weights, args = (self.feature_set, self.squared_sigma, [0]))
        elapsed_time = time.time() - start_time
        print('* Elapsed time: %f' % elapsed_time)
        print('* [%s] CRF Training done' % datetime.datetime.now())
        self.SaveModel()

    def predict(self, filename = None):
        if filename == None:
            filename = self.filename
        test_feature_set = FeatureSet()
        test_feature_set.ReadCorpusCONLL(filename) # 读取语料测试数据集
        totals = corrects = 0
        print('Starting to predict ...')
        for token_poses, labels in test_feature_set.data: # 遍历测试数据集中所有的句子
            labels_predict = self.inference(test_feature_set, token_poses)
            for t, label in enumerate(labels):
                totals += 1
                if label == labels_predict[t]:
                    corrects += 1.0
        print('Corrects: %d' % corrects)
        print('Totals: %d' % totals)
        print('Performance: %f' % (corrects/totals))

    def inference(self, test_feature_set, token_poses):
        sentence_subfeatures = test_feature_set.Featurize(token_poses)
        Mtable = test_feature_set.CalcWeightDotFeature(self.feature_set.subfeature_yy_ids, self.feature_set.label_ids, sentence_subfeatures, self.feature_set.weights)
        return self.Viterbi(sentence_subfeatures, Mtable)

    def Viterbi(self, sentence_subfeatures, Mtable):
        T, M = Mtable.shape[0], Mtable.shape[1]
        max_table, argmax_table = np.zeros((T, M)), np.zeros((T, M), dtype='int64')
        max_table[0] = Mtable[0][0] # 初始化
        for t in range(1, T):
            for id in range(1, M):
                max_value, max_id = -float('inf'), None
                for prev_id in range(1, M):
                    value = max_table[t-1, prev_id] * Mtable[t][prev_id, id]
                    if value > max_value:
                        max_value, max_id = value, prev_id
                        max_table[t, id], argmax_table[t, id] = max_value, max_id
        path = []
        next_id = max_table[T-1].argmax()
        path.append(next_id)
        for t in range(T-1, -1, -1):
            next_id = argmax_table[t, next_id]
            path.append(next_id)
        return [self.feature_set.id_labels[id] for id in path[::-1][1:]]

    def SaveModel(self, filename = None):
        if filename == None:
            filename = os.path.splitext(self.filename)[0] + '.pkl'
        print('* Writing data into file "%s/%s"...' % (os.getcwd(), filename))
        with open(filename, 'wb') as f:
            str = pickle.dumps(self.feature_set)
            f.write(str)
        print('* Trained CRF Model has been saved at "%s/%s"' % (os.getcwd(), filename))

    def LoadModel(self, filename = None):
        if filename == None:
            filename = os.path.splitext(self.filename)[0] + '.pkl'
        print('* Loading file "%s/%s" ...' % (os.getcwd(), filename))
        self.feature_set = FeatureSet()
        with open(filename, 'rb') as f:
            self.feature_set = pickle.loads(f.read())
        print('* Trained CRF Model has been loaded at "%s/%s"' % (os.getcwd(), filename))

### 使用较小的训练数据集进行训练

In [7]:
crf = LinearChainCRF('small_train.data')
crf.fit()

Reading corpus data ...
Data reading completed!
[2020-05-16 10:43:21.045771] Starting to train ...
Step 1 Loss: 5003.65269695053
Step 2 Loss: 4060.008093407453
Step 3 Loss: 1943.6344215443223
Step 4 Loss: 1102.1223123134316
Step 5 Loss: 497.611666890173
Step 6 Loss: 181.5165181898322
Step 7 Loss: 87.99875275067376
Step 8 Loss: 44.3637129975958
Step 9 Loss: 41.05892100271198
Step 10 Loss: 35.40697266317589
Step 11 Loss: 33.359372122616165
Step 12 Loss: 32.11853898282875
Step 13 Loss: 31.40173527607943
Step 14 Loss: 30.715670771585984
Step 15 Loss: 30.066242586500344
Step 16 Loss: 29.586506208383916
Step 17 Loss: 29.32333556653243
Step 18 Loss: 29.20103781679203
Step 19 Loss: 29.10350124146302
Step 20 Loss: 29.071711422692466
Step 21 Loss: 29.029895274449274
Step 22 Loss: 29.018387142109695
Step 23 Loss: 29.000482771868878
Step 24 Loss: 28.983998368451854
Step 25 Loss: 28.988875627548733
Step 26 Loss: 28.97783074786737
Step 27 Loss: 28.971453893003986
Step 28 Loss: 28.96941211891154
Step

### 载入训练好的 CRF 模型，并在较小的测试数据集上进行测试

In [8]:
test_crf = LinearChainCRF('small_test.data')
test_crf.LoadModel('small_train.pkl')
test_crf.predict()

* Loading file "E:\CURRICULUM\Machine Learing\Coding/small_train.pkl" ...
* Trained CRF Model has been loaded at "E:\CURRICULUM\Machine Learing\Coding/small_train.pkl"
Starting to predict ...
Corrects: 17237
Totals: 19172
Performance: 0.899072


### 使用完整的训练数据集进行训练

In [9]:
# crf = LinearChainCRF('full_train.data')
# crf.fit()

## 第 2 个版本（优化）数据集的特征提取类

In [10]:
class FeatureSet:
    def __init__(self):
        # 字典的字典: 存放子特征字符串、前 1 个标签 ID-当前标签 ID、子特征 ID
        self.feature_dict = {}
        # 对子特征的出现次数计数， key：子特征 ID； value：出现次数
        self.empirical_dict = Counter()
        self.num_features = 0 # 子特征个数
        self.squared_sigma = 10.0 # 正则化参数
        self.scale_threshold = 1e250 # 用于计算结果溢出处理
        self.label_dict = {'*': 0} # 开始符号及 ID
        self.label_array = ['*']
    
    # 读取 CONLL 格式的语料文件
    def ReadCorpusCONLL(self, filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
        self.data = [([], [])] # 分别用来存放 TOKEN_POS 序列及其 LABEL
        for line in lines:
            words = line.strip().split()
            if words == []: # 遇到空行
                if self.data[-1] != ([], []):
                    self.data.append(([], []))
            else:
                self.data[-1][0].append(words[:-1]) # 添加 TOKEN 和 POS
                self.data[-1][1].append(words[-1]) # 添加 LABEL
        if self.data[-1] == ([], []):
            del(self.data[-1])
    
    # 为句子生成子特征集，这是缺省的特征函数
    def Featurize(self, X, t):
        length = len(X)
        
        features = []
        features.append('U[0]:%s' % X[t][0])
        features.append('POS_U[0]:%s' % X[t][1])
        if t < length-1:
            features.append('U[+1]:%s' % (X[t+1][0]))
            features.append('B[0]:%s %s' % (X[t][0], X[t+1][0]))
            features.append('POS_U[1]:%s' % X[t+1][1])
            features.append('POS_B[0]:%s %s' % (X[t][1], X[t+1][1]))
            if t < length-2:
                features.append('U[+2]:%s' % (X[t+2][0]))
                features.append('POS_U[+2]:%s' % (X[t+2][1]))
                features.append('POS_B[+1]:%s %s' % (X[t+1][1], X[t+2][1]))
                features.append('POS_T[0]:%s %s %s' % (X[t][1], X[t+1][1], X[t+2][1]))
        if t > 0:
            features.append('U[-1]:%s' % (X[t-1][0]))
            features.append('B[-1]:%s %s' % (X[t-1][0], X[t][0]))
            features.append('POS_U[-1]:%s' % (X[t-1][1]))
            features.append('POS_B[-1]:%s %s' % (X[t-1][1], X[t][1]))
            if t < length-1:
                features.append('POS_T[-1]:%s %s %s' % (X[t-1][1], X[t][1], X[t+1][1]))
            if t > 1:
                features.append('U[-2]:%s' % (X[t-2][0]))
                features.append('POS_U[-2]:%s' % (X[t-2][1]))
                features.append('POS_B[-2]:%s %s' % (X[t-2][1], X[t-1][1]))
                features.append('POS_T[-2]:%s %s %s' % (X[t-2][1], X[t-1][1], X[t][1]))
        return features
    
    # 为语料数据生成所有的子特征及相关数据
    def GenerateAllFeatures(self):
        for X, Y in self.data: # 遍历数据集中的每一条句子
            prev_y = 0 #START 索引 ID 号
            for t in range(len(X)): # 遍历每个 TOKEN， t 表示序列时间步
                # Gets a label id
                try: # 标签 ID 的处理
                    y = self.label_dict[Y[t]]
                except KeyError: # 当前标签 ID 不在 self.label_dict 中，新增 1 个
                    y = len(self.label_dict)
                    self.label_dict[Y[t]] = y
                    self.label_array.append(Y[t])
                # 对当前 TOKEN 的特征进行处理，并统计 Bigram Feature 和
                # Unigram Feature 出现的次数
                self.AddFeature(prev_y, y, X, t)
                prev_y = y #LABEL 索引 ID 号
        self.params = np.zeros(self.num_features)
        self.GenerateEmpiricalCounts()
        self.GenerateAll_YY_IDS()
    
    def AddFeature(self, prev_y, y, X, t):
        for feature_string in self.Featurize(X, t):
            if feature_string in self.feature_dict.keys():
                # 字典的字典：前 1 个标签 ID 和当前标签 ID 作为 key，已存在
                if (prev_y, y) in self.feature_dict[feature_string].keys():
                    self.empirical_dict[self.feature_dict[feature_string][(prev_y, y)]] += 1 #(prev_y,y) 的 ID 号是唯一的
                else: #(prev_y,y) 的 ID 号不存在，创建 1 个
                    feature_id = self.num_features
                    # 生成子特征 ID 号
                    self.feature_dict[feature_string][(prev_y, y)] = feature_id
                    self.empirical_dict[feature_id] = 1
                    self.num_features += 1
                if (-1, y) in self.feature_dict[feature_string].keys():
                    self.empirical_dict[self.feature_dict[feature_string][(-1, y)]] += 1
                else:
                    feature_id = self.num_features
                    self.feature_dict[feature_string][(-1, y)] = feature_id
                    self.empirical_dict[feature_id] = 1
                    self.num_features += 1
            else:
                self.feature_dict[feature_string] = {}
                # Bigram feature
                feature_id = self.num_features
                self.feature_dict[feature_string][(prev_y, y)] = feature_id
                self.empirical_dict[feature_id] = 1
                self.num_features += 1
                # Unigram feature
                feature_id = self.num_features
                self.feature_dict[feature_string][(-1, y)] = feature_id
                self.empirical_dict[feature_id] = 1
                self.num_features += 1
    
    # 计算给定句子的权值-特征的点积（在 K 个特征函数上求和），生成 M 矩阵
    def GenerateMtable(self, params, num_labels, X):
        tables = []
        for t in range(len(X)):
            table = np.zeros((num_labels, num_labels)) # 每个时间步 t 对应的 M 方阵
            for (prev_y, y), feature_ids in X[t]:
                score = sum(params[fid] for fid in feature_ids)
                if prev_y == -1:
                    table[:, y] += score
                else:
                    table[prev_y, y] += score
            table = np.exp(table)
            if t == 0:
                table[1:] = 0
            else:
                table[:, 0] = 0
                table[0, :] = 0
            tables.append(table)
        return tables
    
    # 计算给定句子的 alpha、 beta 与 Z（还包括溢出处理）
    def ForwardBackward(self, num_labels, time_length, potential_table):
        alpha = np.zeros((time_length, num_labels)) #alpha 矩阵
        scaling_dict = {}
        t = 0
        for label_id in range(num_labels): #alpha 前向概率，从 t=0 开始
            alpha[t, label_id] = potential_table[t][0, label_id]
        t = 1
        while t < time_length: # 递推计算前向概率
            scaling_time = None
            scaling_coefficient = None
            overflow_occured = False
            for label_id in range(1, num_labels):
                alpha[t, label_id] = np.dot(alpha[t-1,:], potential_table[t][:,label_id]) # 计算前向概率
                if alpha[t, label_id] > self.scale_threshold:
                    overflow_occured = True
                    scaling_time = t - 1
                    scaling_coefficient = self.scale_threshold
                    scaling_dict[scaling_time] = scaling_coefficient
                    break
            if overflow_occured:
                alpha[t-1] /= scaling_coefficient
                alpha[t] = 0
            else:
                t += 1
        
        beta = np.zeros((time_length, num_labels))
        t = time_length - 1
        for label_id in range(num_labels):
            beta[t, label_id] = 1.0
        for t in range(time_length-2, -1, -1):
            for label_id in range(1, num_labels):
                beta[t, label_id] = np.dot(beta[t+1,:], potential_table[t+1]
                [label_id,:])
                if t in scaling_dict.keys():
                    beta[t] /= scaling_dict[t]
        Z = sum(alpha[time_length-1])
        return alpha, beta, Z, scaling_dict
    
    # 数组：子特征的出现次数
    def GenerateEmpiricalCounts(self):
        self.empirical_counts = np.ndarray((self.num_features,))
        for feature_id, counts in self.empirical_dict.items():
            self.empirical_counts[feature_id] = counts
    
    # 生成给定 TOKEN 的 YY-IDS 对应表
    def GenerateYY_IDS(self, X, t):
        feature_list_dict = {}
        for feature_string in self.Featurize(X, t): # 遍历特征中的每个子特征（字符串）
            try:
                for (prev_y, y), feature_id in self.feature_dict[feature_string].items():
                    if (prev_y, y) in feature_list_dict.keys():
                        feature_list_dict[(prev_y, y)].add(feature_id)
                    else:
                        feature_list_dict[(prev_y, y)] = {feature_id}
            except KeyError: # 应用于测试数据集：可能存在训练数据集中没有的特征
                pass
        return [((prev_y, y), feature_ids) for (prev_y, y), feature_ids in feature_list_dict.items()]
    
    # 所有语料数据集的 YY-IDS 对应表
    def GenerateAll_YY_IDS(self):
        self.training_feature_data = [[self.GenerateYY_IDS(X, t) for t in range(len(X))] for X, Y in self.data]

### 定义 scipy.optimize 的 L-BFGS 函数所需要的回调函数

In [11]:
# 计算数据集的损失函数（负对数似然函数）
def Loss(params, *args):
    feature_set, step = args
    step[0] += 1
    expected_counts = np.zeros(feature_set.num_features)
    total_logZ = 0
    for X_features in feature_set.training_feature_data:
        potential_table = feature_set.GenerateMtable(params,
        len(feature_set.label_dict), X_features)
        alpha, beta, Z, scaling_dict = feature_set.ForwardBackward(len(feature_set.
        label_dict), len(X_features), potential_table)
        # 计算 log(Z1(x)*Z2(x)*...ZT(x))， T 表示序列长度
        total_logZ += log(Z) + sum(log(scaling_coefficient) for _,
        scaling_coefficient in scaling_dict.items())
        for t in range(len(X_features)):
            potential = potential_table[t]
            for (prev_y, y), feature_ids in X_features[t]: # Adds p(prev_y, y | X, t)
                if prev_y == -1:
                    if t in scaling_dict.keys():
                        prob = (alpha[t, y] * beta[t, y] * scaling_dict[t])/Z
                    else:
                        prob = (alpha[t, y] * beta[t, y])/Z
                elif t == 0:
                    if prev_y != 0:
                        continue
                    else:
                        prob = (potential[0, y] * beta[t, y])/Z
                else:
                    if prev_y == 0 or y == 0:
                        continue
                    else:
                        prob = (alpha[t-1, prev_y] * potential[prev_y, y] * beta[t, y]) / Z
                for fid in feature_ids:
                    expected_counts[fid] += prob
    # 数据集的对数似然函数 logP(y1|x1)+...，每个单样本的对数似然函数为： sum(w*f)-logZ，
    # 所有样本的对数似然函数为： sum(sum(w*f))-total_logZ
    # 最后一项是正则化项
    # 带正则化项的对数似然函数
    likelihood = np.dot(feature_set.empirical_counts, params) - total_logZ -np.sum(np.dot(params,params))/(feature_set.squared_sigma*2)
    loss = -likelihood
    # 计算梯度（其中， logZ_sum 对 weights 的梯度，都归结为此项。
    # 先考虑单样本情形， logZ 求偏导 =>F(有效)*P(y|x)，即下面的 prob 项；
    # 再考虑多样本情形即可）
    gradients = -(feature_set.empirical_counts - expected_counts - params/feature_set.squared_sigma) # 负梯度
    print('Step', step[0], 'Loss:', loss)
    return loss, gradients

In [16]:
class LinearChainCRF:
    def __init__(self, filename):
        self.filename = filename
        self.feature_set = FeatureSet()
    
    def fit(self):
        print("* Reading training data ... ", end="")
        self.feature_set.ReadCorpusCONLL(self.filename)
        print("Done")
        self.feature_set.GenerateAllFeatures()
        print("* Number of labels: %d" % (len(self.feature_set.label_array)-1))
        print("* Number of features: %d" % self.feature_set.num_features)
        start_time = time.time()
        print('[%s] Start training' % datetime.datetime.now())
        print('* Squared sigma:', self.feature_set.squared_sigma)
        print('* Start L-BGFS')
        self.feature_set.params, loss, information = fmin_l_bfgs_b(func = Loss,
        x0 = self.feature_set.params, args = (self.feature_set, [0]))
        if information['warnflag'] != 0:
            print('* Warning (code: %d)' % information['warnflag'])
            if 'task' in information.keys():
                print('* Reason: %s' % (information['task']))
        print('* Final loss: %s' % str(loss))
        elapsed_time = time.time() - start_time
        print('* Elapsed time: %f' % elapsed_time)
        print('* [%s] Training done' % datetime.datetime.now())
        self.SaveModel()
    
    def predict(self, filename = None):
        if filename == None:
            filename = self.filename
        test_feature_set = FeatureSet()
        test_feature_set.ReadCorpusCONLL(self.filename)
        total_count = 0
        correct_count = 0
        for X, Y in test_feature_set.data:
            potential_table = self.feature_set.GenerateMtable(self.feature_set.params, len(self.feature_set.label_array), [self.feature_set.GenerateYY_IDS(X, t) for t in range(len(X))])
            Y_HAT = self.Viterbi(X, potential_table)
            for t in range(len(Y)):
                total_count += 1
                if Y[t] == Y_HAT[t]:
                    correct_count += 1
        print('Correct: %d' % correct_count)
        print('Total: %d' % total_count)
        print('Performance: %f' % (correct_count/total_count))
        
    def Viterbi(self, X, potential_table):
        num_labels = len(self.feature_set.label_array)
        time_length = len(X)
        max_table = np.zeros((time_length, num_labels))
        argmax_table = np.zeros((time_length, num_labels), dtype='int64')
        t = 0
        
        for label_id in range(num_labels):
            max_table[t, label_id] = potential_table[t][0, label_id]
        for t in range(1, time_length):
            for label_id in range(1, num_labels):
                max_value = -float('inf')
                max_label_id = None
                for prev_label_id in range(1, num_labels):
                    value = max_table[t-1, prev_label_id] * potential_table[t][prev_label_id, label_id]
                    if value > max_value:
                        max_value = value
                        max_label_id = prev_label_id
                max_table[t, label_id] = max_value
                argmax_table[t, label_id] = max_label_id
        sequence = []
        next_label = max_table[time_length-1].argmax()
        sequence.append(next_label)
        for t in range(time_length-1, -1, -1):
            next_label = argmax_table[t, next_label]
            sequence.append(next_label)
        return [self.feature_set.label_array[label_id] for label_id in sequence[::-1][1:]]

    def SaveModel(self, filename = None):
        if filename == None:
            filename = os.path.splitext(self.filename)[0] + '_new' + '.pkl'
        print('* Writing data into file "%s/%s"...' % (os.getcwd(), filename))
        with open(filename, 'wb') as f:
            str = pickle.dumps(self.feature_set)
            f.write(str)
        print('* Trained CRF Model has been saved at "%s/%s"' % (os.getcwd(), filename))
    
    def LoadModel(self, filename = None):
        if filename == None:
            filename = os.path.splitext(self.filename)[0] + '.pkl'
        print('* Loading file "%s/%s" ...' % (os.getcwd(), filename))
        self.feature_set = FeatureSet()
        with open(filename, 'rb') as f:
            self.feature_set = pickle.loads(f.read())
        print('* Trained CRF Model has been loaded at "%s/%s"' % (os.getcwd(), filename))

### 使用较小的训练数据集进行训练

In [17]:
crf = LinearChainCRF('small_train.data')
crf.fit()

* Reading training data ... Done
* Number of labels: 14
* Number of features: 31018
[2020-05-16 10:45:53.049913] Start training
* Squared sigma: 10.0
* Start L-BGFS
Step 1 Loss: 5003.65269695053
Step 2 Loss: 4060.008093407453
Step 3 Loss: 1943.6344215443223
Step 4 Loss: 1102.1223123134298
Step 5 Loss: 497.611666890173
Step 6 Loss: 181.5165181898304
Step 7 Loss: 87.99875275067197
Step 8 Loss: 44.363712997597624
Step 9 Loss: 41.05892100271201
Step 10 Loss: 35.40697266317016
Step 11 Loss: 33.359372122623405
Step 12 Loss: 32.118538982828944
Step 13 Loss: 31.401735276072326
Step 14 Loss: 30.715670771586105
Step 15 Loss: 30.06624258650195
Step 16 Loss: 29.586506208385874
Step 17 Loss: 29.323335566521976
Step 18 Loss: 29.20103781679533
Step 19 Loss: 29.103501241464198
Step 20 Loss: 29.071711422686956
Step 21 Loss: 29.029895274451302
Step 22 Loss: 29.018387142113443
Step 23 Loss: 29.000482771868782
Step 24 Loss: 28.983998368453463
Step 25 Loss: 28.988875627545717
Step 26 Loss: 28.9778307478676

In [18]:
test_crf = LinearChainCRF('small_test.data')
test_crf.LoadModel('small_train_new.pkl')
test_crf.predict()

* Loading file "E:\CURRICULUM\Machine Learing\Coding/small_train_new.pkl" ...
* Trained CRF Model has been loaded at "E:\CURRICULUM\Machine Learing\Coding/small_train_new.pkl"
Correct: 17237
Total: 19172
Performance: 0.899072


### 使用完整的训练数据集进行训练

In [15]:
# crf = LinearChainCRF('full_train.data')
# crf.fit()